# Script sparcing informe Rettig

## 1. pasar de pdf a html todo el archivo

USé este convertidor de pdf a html. Es un wraper de pdfminer pa pasarlo a Html

https://gist.github.com/rguliev/3d886d38daa8ac0be8ddb85d645fb0bc#file-convert_pdf-py

In [23]:
from typing import Container
from io import BytesIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import HTMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert_pdf_to_html(path: str, start_page: int, end_page: int) -> str:
    """Converts a range of pages from a PDF file to HTML format."""
    rsrcmgr = PDFResourceManager()
    retstr = BytesIO()
    laparams = LAParams()
    device = HTMLConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, "rb")
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page_num, page in enumerate(PDFPage.get_pages(fp), start=1):
        if page_num < start_page:
            continue
        if page_num > end_page:
            break
        interpreter.process_page(page)
    html = retstr.getvalue().decode()
    fp.close()
    device.close()
    retstr.close()
    return html

# Define the path to the PDF file and the range of pages to extract
pdf_file_path = "Informe-Rettig-tomo1.pdf"
start_page = 139
end_page = 252

# Convert PDF pages to HTML
html_content = convert_pdf_to_html(pdf_file_path, start_page, end_page)

print("html object created")

html object created


In [27]:
# exportamos el html para examinar el archivo 

def export_html_to_file(html_content: str, output_file: str):
    """Exports HTML content to a file."""
    with open(output_file, "w", encoding="utf-8") as file:
        file.write(html_content)

# Define the path for the output HTML file
output_html_file = "data_out/output.html"

# Export HTML content to the output HTML file
export_html_to_file(html_content, output_html_file)
 
print(f"HTML content has been exported to {output_html_file}.")


HTML content has been exported to data_out/output.html.


In [37]:
print(html_content)

<html><head>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
</head><body>
<span style="position:absolute; border: gray 1px solid; left:0px; top:50px; width:612px; height:792px;"></span>
<div style="position:absolute; top:50px;"><a name="1">Page 1</a></div>
<div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:210px; top:125px; width:293px; height:40px;"><span style="font-family: TimesNewRoman; font-size:9px">Entre los recintos de detención usados durante 1973 en la Región Metropolitana, 
<br>se  encuentra  el  local  de  calle  Londres  38,  que  antes  del  11  de  septiembre 
<br>perteneció  al  Partido  Socialista,  y  que  más  tarde  sería  usado  como  centro 
<br>clandestino de reclusión por la DINA. 
<br></span></div><div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:210px; top:178px; width:293px; height:83px;"><span style="font-family: TimesNewRoman; font-size:9px">Se dispone de antecedentes que 

## 2. maniulacion del HTML para tener una base de datos

### 1. Extracción de items en negritas para obtencion de lista de nombres.

Usamos beautiful soup para sparcer de html

In [24]:
from bs4 import BeautifulSoup

def extract_names(html_content):
    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all bold elements with text inside them
    bold_elements = soup.find_all('span', style=lambda value: value and 'font-family: TimesNewRoman,Bold' in value and len(value) > 0)

    # Extract names
    names = []
    current_name = ""
    for bold_element in bold_elements:
        text = bold_element.get_text().strip()

        # Append the current name to the list of names if it's not empty
        if current_name:
            names.append(current_name)
            current_name = ""

        # Check if the bold element is followed by another tag
        if bold_element.find_next_sibling():
            # Append the text to the list of names
            names.append(text)
        else:
            # Store the text as the current name
            current_name = text

    return names

# Extract names
names = extract_names(html_content)


In [25]:
import re


# Filter out names without capital letters
filtered_names = [name for name in names if any(char.isupper() for char in name)]


# Function to filter out symbols that are not letters from a string
def filter_symbols(name):
    return re.sub(r'[,.-]', '', name)

# Filter out symbols from each name in the list
cleaned_names = [filter_symbols(name) for name in filtered_names]




In [34]:

def concatenate_names(names):
    concatenated_names = []
    prev_name = None

    for name in names:
        if name.isupper():
            if prev_name is not None:
                concatenated_name = prev_name + " " + name
                concatenated_names[-1] = concatenated_name
            else:
                concatenated_names.append(name)
        else:
            concatenated_names.append(name)
            prev_name = name
    
    return concatenated_names

# Concatenate names with all uppercase letters and a space with the previous item
concatenated_names = concatenate_names(cleaned_names)

concatenated_names = [name for name in concatenated_names if name != "Lonquén"]
name_list = concatenated_names[1:]


In [35]:
name_list = pd.DataFrame({"names": name_list})
name_list.to_csv('data_out/names_rettig.csv')

name_list


,names
0,Salvador ALLENDE GOSSENS
1,Antonio AGUIRRE VASQUEZ
2,Osvaldo RAMOS \nRIVERA
3,Augusto OLIVARES BECERRA
4,Jaime BARRIOS MEZA
...,...
490,Luis Heriberto CONTRERAS \nESCANILLA
491,Rudy Freddy VIDAL PEREIRA
492,Manuel Tomás ROJAS FUENTES
493,Juan Domingo MARTINEZ ALDANA


### 2. extraccion lista de casos por italicas 

In [ ]:
from bs4 import BeautifulSoup

def extract_events(html_content):
    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Remove all <br> tags
    for br in soup.find_all('br'):
        br.unwrap()
    
    # Find all italic elements containing the date
    italic_elements = soup.find_all('span', style=lambda value: value and 'font-family: TimesNewRoman,Italic' in value)
    
    # Extract events based on the italicized dates
    events = []
    for italic_element in italic_elements:
        # Find the parent div containing the italic element
        parent_div = italic_element.find_parent('div')
        
        if parent_div:
            # Initialize event text
            event_text = ""
            
            # Iterate over the following sibling divs and spans
            next_sibling = parent_div.find_next_sibling()
            while next_sibling:
                # Check if the sibling is another italicized element
                if 'Italic' in next_sibling.get('style', ''):
                    break
                
                # Append the text of the sibling element to the event text
                event_text += next_sibling.get_text().strip() + " "
                
                # Move to the next sibling
                next_sibling = next_sibling.find_next_sibling()
            
            # Append the extracted event text to the list of events
            events.append(event_text.strip())
    
    return events

# Extract events from the HTML content
events = extract_events(html_content)

# Print the extracted events
for i, event in enumerate(events, start=1):
    print(f"Event {i}:")
    print(event)
    print("---")

    


In [48]:

events_list = pd.DataFrame({"events": events})
events_list.to_csv('data_out/events_rettig.csv')



In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

def extract_events(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    events = []
    current_event = ""
    current_date = ""
    bold_names = []

    # Find all italic elements preceded by a paragraph break
    italic_elements = soup.find_all('span', style=lambda value: value and 'font-family: TimesNewRoman,Italic' in value)
    for italic_element in italic_elements:
        # Check if the previous element is a paragraph break
        prev_sibling = italic_element.find_previous_sibling()
        if prev_sibling and prev_sibling.name == 'br':
            # Extract the date
            current_date = italic_element.text.strip()
            current_event = ""

            # Get next sibling
            next_sibling = italic_element.find_next_sibling()

            # Loop through siblings until the next italic element is found
            while next_sibling and (not next_sibling.name == 'span' or 'font-family: TimesNewRoman,Italic' not in next_sibling.get('style', '')):
                # Check if the sibling is a bold element
                if next_sibling.name == 'span' and 'font-family: TimesNewRoman,Bold' in next_sibling.get('style', ''):
                    bold_text = next_sibling.text.strip()
                    if bold_text:
                        bold_names.append(bold_text)
                
                # Add text to the current event
                if next_sibling.name == 'br':
                    current_event += '\n'
                elif next_sibling.name is None:
                    current_event += next_sibling.strip()

                # Move to the next sibling
                next_sibling = next_sibling.find_next_sibling()

            # Append the event to the list of events
            events.append({
                "date": current_date,
                "event": current_event.strip(),
                "names": ", ".join(bold_names)
            })
            bold_names = []

    return pd.DataFrame(events)


# Extract events from the HTML content
df_events = extract_events(html_content)

print(df_events)


In [40]:
from bs4 import BeautifulSoup

def extract_events(html_content):
    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find all italic elements containing the date
    italic_elements = soup.find_all('span', style=lambda value: value and 'font-family: TimesNewRoman,Italic' in value)
    
    # Extract events based on the italicized dates
    events = []
    for italic_element in italic_elements:
        # Check if the italic element is preceded by a paragraph break
        if italic_element.previous_sibling and italic_element.previous_sibling.name == 'br':
            # Initialize event text
            event_text = ""
            
            # Find the next sibling elements until the next italic element
            next_sibling = italic_element.find_next_sibling()
            while next_sibling and 'Italic' not in next_sibling.get('style', ''):
                # Append the text of the sibling element to the event text
                event_text += next_sibling.get_text().strip() + " "
                next_sibling = next_sibling.find_next_sibling()
            
            # Append the extracted event text to the list of events
            events.append({
                "date": italic_element.get_text().strip(),
                "event": event_text.strip()
            })
    
    return events


# Extract events from the HTML content
events = extract_events(html_content)

# Print the extracted events
for event in events:
    print(f"Date: {event['date']}")
    print(f"Event: {event['event']}")
    print("---")


### 3. summary

In [42]:
import pandas as pd
from bs4 import BeautifulSoup

def extract_events_and_names(html_content):
    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Remove all <br> tags
    for br in soup.find_all('br'):
        br.unwrap()
    
    # Find all italic elements containing the date
    italic_elements = soup.find_all('span', style=lambda value: value and 'font-family: TimesNewRoman,Italic' in value)
    
    # Extract events and associated names based on the italicized dates
    events = []
    names = []
    for italic_element in italic_elements:
        # Find the parent div containing the italic element
        parent_div = italic_element.find_parent('div')
        
        if parent_div:
            # Initialize event text and names for the current event
            event_text = ""
            event_names = []
            
            # Iterate over the following sibling divs and spans
            next_sibling = parent_div.find_next_sibling()
            while next_sibling:
                # Check if the sibling is another italicized element
                if 'Italic' in next_sibling.get('style', ''):
                    break
                
                # Append the text of the sibling element to the event text
                event_text += next_sibling.get_text().strip() + " "
                
                # Find bold elements inside the sibling
                bold_elements = next_sibling.find_all('span', style=lambda value: value and 'font-family: TimesNewRoman,Bold' in value)
                for bold_element in bold_elements:
                    event_names.append(bold_element.get_text().strip())
                
                # Move to the next sibling
                next_sibling = next_sibling.find_next_sibling()
            
            # Append the extracted event text to the list of events
            events.append(event_text.strip())
            
            # Append the extracted names to the list of names
            names.append(", ".join(event_names))
    
    return events, names

# Extract events and associated names from the HTML content
events, names = extract_events_and_names(html_content)

# Create a DataFrame
df = pd.DataFrame({
    "Event": events,
    "Names": names
})

# Display the DataFrame
print(df)


                                                 Event  \
0    Alrededor de las 13:00 horas, una vez que ya s...   
1    De  acuerdo  a  estos  antecedentes,  la  Comi...   
2    El  día  indicado,  por  la  mañana,  se  diri...   
3    De acuerdo a los testimonios recibidos por la ...   
4    Con  los  demás  detenidos  junto  a  él  es  ...   
..                                                 ...   
276  La  Comisión  se  formó  convicción  de  que  ...   
277  El  10  de  noviembre  fue  detenido  Contrera...   
278  En  la  madrugada  de  ese  día  llegó  a  su ...   
279  Manuel  Tomás  ROJAS  FUENTES,  de  20  años, ...   
280  El  afectado  se  había  reincorporado  a  ést...   

                                                 Names  
0    Salvador ALLENDE GOSSENS, Antonio  AGUIRRE  VA...  
1    Manuel OJEDA \nDISSELKOEN,, Jorge Claudio \nAR...  
2    Jorge Claudio \nARAVENA  MARDONES, Guillermo J...  
3    Guillermo Jesús ARENAS DIAZ, José Agustín FARF...  
4                 